## Project Polling Feature

In [ ]:
import syft as sy
sy.requires(">=0.8.1-beta")
sy.enable_autoreload()

from syft.service.project.project import ProjectMultipleChoicePoll, AnswerProjectPoll

In [ ]:
alice_domain = sy.orchestra.launch(name="Alice", dev_mode=True)
alice_domain_client = alice_domain.login(email="info@openmined.org", password="changethis")

In [ ]:
bob_domain = sy.orchestra.launch(name="Bob", dev_mode=True)
bob_domain_client = bob_domain.login(email="info@openmined.org", password="changethis")

## Retrieving existing projects

In [ ]:
alice_projects = alice_domain_client.api.services.project.get_all()
alice_project = alice_projects[0]
assert len(alice_projects)==1
alice_projects

In [ ]:
bob_projects = bob_domain_client.api.services.project.get_all()
bob_project = bob_projects[0]
assert len(bob_projects)==1
bob_projects

In [ ]:
assert alice_project.user_signing_key == alice_domain_client.api.signing_key

In [ ]:
assert bob_project.user_signing_key == bob_domain_client.api.signing_key

### 1. Initial Creation Poll by Alice

In [ ]:
question = "Do you like StarWars ⚔️ or StarTrek 🛫 "
choices = ["StarWars","StarTrek"]
alice_project.create_poll(question=question, choices= choices)

In [ ]:
poll = alice_project.events[5]
assert isinstance(poll, ProjectMultipleChoicePoll)

In [ ]:
assert poll.question ==question
assert poll.choices == choices

### 2. Alice answering the Poll

In [ ]:
alice_poll_answer = 1
alice_project.answer_poll(answer=alice_poll_answer, poll=alice_project.events[5].id)

In [ ]:
alice_poll = alice_project.events[6]
assert isinstance(alice_poll, AnswerProjectPoll)

In [ ]:
assert alice_poll.answer == alice_poll_answer

### 3. Bob answering the Poll

In [ ]:
bob_project.sync()

In [ ]:
bob_poll_answer = 2
bob_project.answer_poll(answer= bob_poll_answer, poll=bob_project.events[5])

In [ ]:
bob_poll = bob_project.events[7]
assert isinstance(bob_poll, AnswerProjectPoll)

In [ ]:
assert bob_poll.answer == bob_poll_answer

### 4. Poll Results

In [ ]:
alice_project.sync()

In [ ]:
poll_results = alice_project.events[5].status(alice_project, pretty_print=False)

In [ ]:
shareholders = alice_project.members
alice_identity = alice_project.get_identity_from_key(verify_key = alice_domain_client.credentials.verify_key)

In [ ]:
assert alice_identity in poll_results
assert poll_results[alice_identity] == alice_poll_answer

In [ ]:
bob_identity = bob_project.get_identity_from_key(verify_key = bob_domain_client.credentials.verify_key)

In [ ]:
assert bob_identity in poll_results
assert poll_results[bob_identity] == bob_poll_answer

### 5. Change of Poll Answers

If Alice and Bob change their mind on the casted vote, and they would like to modify the answers

In [ ]:
alice_new_poll_answer = 2
alice_project.answer_poll(answer=alice_new_poll_answer, poll=alice_project.events[5])

In [ ]:
bob_new_poll_answer = 1
bob_project.answer_poll(answer= bob_new_poll_answer, poll=bob_project.events[5].id)

### 6. New Poll Results

In a poll, only the latest casted vote is taken into consideration

In [ ]:
alice_project.sync()

In [ ]:
new_poll_results = alice_project.events[5].status(alice_project, pretty_print=False)

In [ ]:
assert alice_identity in new_poll_results
assert new_poll_results[alice_identity] == alice_new_poll_answer

In [ ]:
assert bob_identity in new_poll_results
assert new_poll_results[bob_identity] == bob_new_poll_answer

In [ ]:
res =  alice_project.validate_events(debug=True)
res

In [ ]:
assert isinstance(res, sy.SyftSuccess)

In [ ]:
res =  bob_project.validate_events(debug=True)
res

In [ ]:
assert isinstance(res, sy.SyftSuccess)

In [ ]:
for node in [alice_domain, bob_domain]:
    if node.node_type.value == "python":
        node.land()